In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [2]:
# data_type = "train"
# month = "07"
# category = "잔액정보"

# local
root_path = '../../data'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

#drive_folder = f'{root_path}/스탯티즈_Raw_data/'

In [3]:
train = pd.read_csv(f'{root_path}/KBO FA 투수 [2013-2024]_최종_전처리_ver03.csv', encoding='utf-8')

In [4]:
# 1. AAV 생성 및 결측치 제거
train["AAV"] = train["FA 계약 총액"] / train["FA 계약 연수"]
train = train.dropna(subset=["AAV"])

In [5]:
# 2. 필요 없는 컬럼 제거
drop_cols = [
    'FA 계약 총액', 'FA년도', '구단명', 'FA등급',
    '비고1', '구분', '잔류 여부', '구종1', '구종2', '구종1 구종가치'
]
train = train.drop(columns=drop_cols, errors='ignore')

In [6]:
# 3. 선수별 수치 피처 평균 요약
train_grouped = train.groupby('선수명').mean(numeric_only=True).reset_index()

In [7]:
# 4. 범주형 변수 원핫 인코딩 후 병합
cat_cols = ['투', '세부 포지션']
for col in cat_cols:
    if col in train.columns:
        temp = pd.get_dummies(train[[col, '선수명']], columns=[col])
        temp = temp.groupby('선수명').max().reset_index()
        train_grouped = pd.merge(train_grouped, temp, on='선수명', how='left')

In [8]:
# 6. 피처 / 타겟 정의
X = train_grouped.drop(columns=['AAV', '선수명'])
y = train_grouped['AAV']

In [9]:
# 🎯 Dummy 데이터 생성 및 결합 (선수 단위 평균 이후 삽입)

# 수치형 feature만 추출 (AAV 제외)
feature_cols = train_grouped.select_dtypes(include=['float64', 'int64']).drop(columns=['AAV'], errors='ignore').columns
X_real = train_grouped[feature_cols]
y_real = train_grouped['AAV']

In [33]:
# dummy 100개 생성: 평균 ± 표준편차 기반
np.random.seed(42)
num_dummy = 10
X_dummy = pd.DataFrame(
    np.random.randn(num_dummy, len(feature_cols)) * X_real.std().values + X_real.mean().values,
    columns=feature_cols
)
y_dummy = np.random.uniform(low=0.5, high=13.0, size=num_dummy)  # 단위: 억

In [35]:
# 실제 + dummy 결합
X_all = pd.concat([X_real, X_dummy], ignore_index=True)
y_all = pd.concat([y_real, pd.Series(y_dummy)], ignore_index=True)

print(f"📌 실제 데이터 {len(X_real)}개 + Dummy {num_dummy}개 → 총 학습 데이터: {len(X_all)}개")

📌 실제 데이터 66개 + Dummy 10개 → 총 학습 데이터: 76개


In [37]:
# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(
    X_all, y_all,
    test_size=0.2,       # 검증 데이터 20%
    random_state=42      # 재현성
)

# 확인
print("학습 데이터:", X_train.shape, "검증 데이터:", X_val.shape)

학습 데이터: (60, 40) 검증 데이터: (16, 40)


In [39]:
# 모델 정의 및 학습
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [40]:
# 예측
y_pred = model.predict(X_val)

In [43]:
# 평가 지표
rmse = mean_squared_error(y_val, y_pred, squared=False)
mes = np.mean(y_pred - y_val)
r2 = r2_score(y_val, y_pred)

print(f"✅ RMSE: {rmse:,.0f} 원")
print(f"✅ MES: {mes:.4f}")
print(f"✅ R²: {r2:.4f}")

✅ RMSE: 36,359 원
✅ MES: 22390.5508
✅ R²: 0.2006
